## MindNLP ChatGLM-6B StreamChat

本案例基于MindNLP和ChatGLM-6B实现一个聊天应用。**支持流式回复**。

该实验可进行在线体验，在线体验链接（https://pangu.huaweicloud.com/gallery/asset-detail.html?id=cdc88c83-1ac2-4862-b822-3ab200b01736
）。

## 1. 效果展示


<video controls width="500" height="400" src="https://mindspore-demo.obs.cn-north-4.myhuaweicloud.com/imgs/musicgen/chatglm6b-streamchat-demo.mp4">animation</video>

## 2. 案例体验
🔹 本案例需使用 P100 及以上规格运行，请确保运行规格一致，可按照下图切换规格。

![](https://modelarts-labs-bj4-v2.obs.cn-north-4.myhuaweicloud.com/case_zoo/chatglm3/image/1.png)

🔹 点击Run in ModelArts，将会进入到ModelArts CodeLab中，这时需要你登录华为云账号，如果没有账号，则需要注册一个，且要进行实名认证，参考[《ModelArts准备工作_简易版》](https://developer.huaweicloud.com/develop/aigallery/article/detail?id=4ce709d6-eb25-4fa4-b214-e2e5d6b7919c) 即可完成账号注册和实名认证。 登录之后，等待片刻，即可进入到CodeLab的运行环境

🔹 出现 Out Of Memory ，请检查是否为您的参数配置过高导致，修改参数配置，重启kernel或更换更高规格资源进行规避❗❗❗

### 2.1 环境安装


运行如下两个代码块，创建python-3.9.0环境并在notebook的kernel显示选项。

> 注意：
>
> 此为在线运行平台配置python3.9的指南，如在其他环境平台运行案例，请根据实际情况修改如下代码。
>
> 以下两个代码块仅能运行一次，多次运行会出现kernel报错。
>
> 如出现多次运行导致的kernel报错，请终止实例（点击右上角“停止NoteBook实例”的圆形图标），并重启实例。

In [ ]:
%%capture captured_output
!/home/ma-user/anaconda3/bin/conda create -n python-3.9.0 python=3.9.0 -y --override-channels --channel https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
!/home/ma-user/anaconda3/envs/python-3.9.0/bin/pip install ipykernel

In [ ]:
import json
import os

data = {
   "display_name": "python-3.9.0",
   "env": {
      "PATH": "/home/ma-user/anaconda3/envs/python-3.9.0/bin:/home/ma-user/anaconda3/envs/python-3.7.10/bin:/modelarts/authoring/notebook-conda/bin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/ma-user/modelarts/ma-cli/bin:/home/ma-user/modelarts/ma-cli/bin"
   },
   "language": "python",
   "argv": [
      "/home/ma-user/anaconda3/envs/python-3.9.0/bin/python",
      "-m",
      "ipykernel",
      "-f",
      "{connection_file}"
   ]
}

if not os.path.exists("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/"):
    os.mkdir("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/")

with open('/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/kernel.json', 'w') as f:
    json.dump(data, f, indent=4)

创建完成后，稍等片刻，或刷新页面，点击右上角（或左上角）kernel选择python-3.9.0

![change-kernel](https://mindspore-demo.obs.cn-north-4.myhuaweicloud.com/imgs/ai-gallery/change-kernel.PNG)

In [ ]:
%%capture captured_output
pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.4.10/MindSpore/unified/aarch64/mindspore-2.4.10-cp39-cp39-linux_aarch64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install mindnlp

In [ ]:
!pip install gradio mdtex2html -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install ipywidgets -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
# %%capture captured_output
!wget -P /home/ma-user/work https://openi.pcl.ac.cn/lvyufeng/frpc-gradio/raw/branch/master/frpc_linux_amd64
!cp /home/ma-user/work/frpc_linux_amd64 /home/ma-user/anaconda3/envs/python-3.9.0/lib/python3.9/site-packages/gradio/frpc_linux_amd64_v0.2
!chmod +x /home/ma-user/anaconda3/envs/python-3.9.0/lib/python3.9/site-packages/gradio/frpc_linux_amd64_v0.2

## 3. 代码开发

In [ ]:
from mindnlp.transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import gradio as gr
import mdtex2html
import mindspore

# Ascend平台请取消以下注释，使用动态图模式
mindspore.set_context(mode=mindspore.context.PYNATIVE_MODE,device_target='Ascend')

model = AutoModelForSeq2SeqLM.from_pretrained('MindSpore-Lab/ChatGLM-6B', mirror="modelers").half()
model.set_train(False)
tokenizer = AutoTokenizer.from_pretrained('MindSpore-Lab/ChatGLM-6B', mirror="modelers")

In [ ]:
prompt = '你好'
history = []
response, _ = model.chat(tokenizer, prompt, history=history, max_length=20)
response

In [ ]:
import os
from typing import Dict, Union, Optional
from typing import List


import mdtex2html

#将文本中的字符转为网页上可以支持的字符，避免被误认为是HTML标签
def parse_text(text):
    """copy from https://github.com/GaiZhenbiao/ChuanhuChatGPT/"""
    lines = text.split("\n")
    lines = [line for line in lines if line != ""]
    count = 0
    for i, line in enumerate(lines):
        if "```" in line:
            count += 1
            items = line.split('`')
            if count % 2 == 1:
                lines[i] = f'<pre><code class="language-{items[-1]}">'
            else:
                lines[i] = f'<br></code></pre>'
        else:
            if i > 0:
                if count % 2 == 1:
                    line = line.replace("`", "\`")
                    line = line.replace("<", "&lt;")
                    line = line.replace(">", "&gt;")
                    line = line.replace(" ", "&nbsp;")
                    line = line.replace("*", "&ast;")
                    line = line.replace("_", "&lowbar;")
                    line = line.replace("-", "&#45;")
                    line = line.replace(".", "&#46;")
                    line = line.replace("!", "&#33;")
                    line = line.replace("(", "&#40;")
                    line = line.replace(")", "&#41;")
                    line = line.replace("$", "&#36;")
                lines[i] = "<br>"+line
    text = "".join(lines)
    return text

## 3.2 基于 Gradio 创建聊天应用

In [ ]:
# 编写Gradio调用函数

#采用流聊天方式（stream_chat）调用模型，使得生成答案有逐字生成的效果
def predict(input, chatbot, max_length, top_p, temperature, history):
    chatbot.append((parse_text(input), ""))
    for response, history in model.stream_chat(tokenizer, input, history, max_length=max_length, top_p=top_p,
                                               temperature=temperature):
        chatbot[-1] = (parse_text(input), parse_text(response))       

        yield chatbot, history

#去除输入框的内容
def reset_user_input():
    return gr.update(value='')

#清除状态
def reset_state():
    return [], [], None

In [ ]:
#运行Gradio界面，运行成功后点击“Running on public URL”后的网页链接即可体验
import gradio as gr

with gr.Blocks() as demo:
    gr.HTML("""<h1 align="center">MindNLP ChatGLM-6B StreamChat</h1>""")

    chatbot = gr.Chatbot()
    with gr.Row():
        with gr.Column(scale=4):
            with gr.Column(scale=12):
                user_input = gr.Textbox(show_label=False, placeholder="Input...", lines=3, container=False)
            with gr.Column(min_width=32, scale=1):
                with gr.Row():
                    submitBtn = gr.Button("一键开聊", variant="primary")
                    emptyBtn = gr.Button("清除历史")
            with gr.Column(scale=1):
                max_length = gr.Slider(0, 4096, value=2048, step=1.0, label="Maximum length", interactive=True)
                top_p = gr.Slider(0, 1, value=0.7, step=0.01, label="Top P", interactive=True)
                temperature = gr.Slider(0, 1, value=0.95, step=0.01, label="Temperature", interactive=True)               

    history = gr.State([])

    submitBtn.click(predict, [user_input, chatbot, max_length, top_p, temperature, history], [chatbot, history],
                    show_progress=True)
    submitBtn.click(reset_user_input, [], [user_input])

    emptyBtn.click(reset_state, outputs=[chatbot, history], show_progress=True)

demo.queue().launch(share=True)